<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer: The input layer receives the inputs into the model and is visible to the user. It is also able to be manipulated by the user.

### Hidden Layer: The hidden layers are the middle layers of the model which are not visible to the user and are not able to be manipulated by the user.

### Output Layer: The outer layer outputs a vector with values in a format that are relevant to the problem at hand. For a binary classification problem, for example, the output would be in the format of values between 0 and 1.

### Neuron: It is the most elementary unit of a neural network. It receives input values, multiplies them by their weights, sums them up and then applies the activation function to the sum.

### Weight: A measure of strength or impact of one node on the node in the next layer. In other words, the amount of influence the firing of one neuron has on another.

### Activation Function: Defines the output of a node given a set of inputs.

### Node Map: A diagram showing the paths from inputs to outputs in a neural network.

### Perceptron: A single-layered neural network.


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### A neural network is composed of a series of layers. The data is received first by the input layer. The inputs are then multiplied by their weights, which determines how much importance the inputs have on the outputs. These are then summed together and passed through an activation function, which determines whether the weighted sum is greater than a certain threshold value--equivalent to the bias value--and as a result, is then assigned either a 1 or 0 as an output

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import numpy as np

np.random.seed(42)

inputs = np.array([
    [0,0,1],
    [1,0,1],
    [0,1,1],
    [1,1,1]
])

correct_outputs = [[1], [1], [1], [0]]

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [3]:
weights = 2 * np.random.random((3,1)) - 1
weights

array([[-0.25091976],
       [ 0.90142861],
       [ 0.46398788]])

In [4]:
weighted_sum = np.dot(inputs, weights)
weighted_sum

array([[0.46398788],
       [0.21306812],
       [1.3654165 ],
       [1.11449673]])

In [5]:
activated_output = sigmoid(weighted_sum)
activated_output

array([[0.61395979],
       [0.55306642],
       [0.79663861],
       [0.75296649]])

In [6]:
error = correct_outputs - activated_output
error

array([[ 0.38604021],
       [ 0.44693358],
       [ 0.20336139],
       [-0.75296649]])

In [7]:
adjustments = error * sigmoid_derivative(activated_output)
adjustments

array([[ 0.08795756],
       [ 0.10360651],
       [ 0.04355646],
       [-0.16389299]])

In [8]:

weights += np.dot(inputs.T, adjustments)
weights

array([[-0.31120624],
       [ 0.78109208],
       [ 0.53521543]])

In [9]:
for iteration in range(10000):
    
    #weighted sum of inputs / weights
    weighted_sum = np.dot(inputs, weights)
    
    #activate
    activated_output = sigmoid(weighted_sum)
    
    #cac error
    error = correct_outputs - activated_output
    
    adjustments = error * sigmoid_derivative(activated_output)
    
    #update the weights
    
    weights += np.dot(inputs.T, adjustments)
    
print("weights after training")
print(weights)
    
print("output after training")
print(activated_output)

weights after training
[[-11.83951525]
 [-11.83951525]
 [ 17.80813756]]
output after training
[[0.99999998]
 [0.99744851]
 [0.99744851]
 [0.0028127 ]]


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
X = df.drop(columns=['Outcome']).values
y = df['Outcome'].values

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((514, 8), (254, 8), (514,), (254,))

In [17]:
class Perceptron(object):
  def __init__(self, rate = 0.01, niter = 10):
    self.rate = rate
    self.niter = niter

  def fit(self, X, y):
    """Fit training data
    X : Training vectors, X.shape : [#samples, #features]
    y : Target values, y.shape : [#samples]
    """

    # weights
    self.weight = np.zeros(1 + X.shape[1])

    # Number of misclassifications
    self.errors = []  # Number of misclassifications

    for i in range(self.niter):
      err = 0
      for xi, target in zip(X, y):
        delta_w = self.rate * (target - self.predict(xi))
        self.weight[1:] += delta_w * xi
        self.weight[0] += delta_w
        err += int(delta_w != 0.0)
      self.errors.append(err)
    return self

  def net_input(self, X):
    """Calculate net input"""
    return np.dot(X, self.weight[1:]) + self.weight[0]

  def predict(self, X):
    """Return class label after unit step"""
    return np.where(self.net_input(X) >= 0.0, 1, 0)


In [19]:
perceptron = Perceptron(0.01, 10000)

perceptron.fit(X_train, y_train)

predictions = [perceptron.predict(X_test[i]) for i in range(len(X_test))]
print("accuracy:", accuracy_score(y_test, predictions))

accuracy: 0.7401574803149606


## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?